Toastmasters scheduling with MIP
---

Try to schedule all the rules of Toastmasters using a MIP approach.

The restrictions are:
- everyone has to do each of the three "main" jobs once
- the three main jobs are repeated 2 or 3 times on a single day
- students should speak "against" a different set of speakers between their two times
- auxiliary jobs should be filled with remaining students, with preference to students who haven't done that job yet

In [1]:
import pulp as pl
import numpy as np
import os

In [24]:
students2025 = """Hiro
Toka
Rene
Nene
Yuna H.
Rui
Yuna C.
Kazu
Himi
Kokoro
Aika
Sara
Risako
Hanna
Suzuna
Yuriko
George
Yamato
Daisy
Koriki
Miharu
Taisuke""".split('\n')

jobs = {
    "prepared_roles":['prepared1','prepared2','prepared3'],
    "impromptu_roles" :['impromptu1','impromptu2','impromptu3'],
    "evaluator_roles" :['evaluator1','evaluator2','evaluator3'],
    "leadership" : [
        "President",
        "Toastmaster",
        "Table Topics Master",
        "General Evaluator"
    ],
    "aux":[
        "Greeter",
        "Joke Master",
        "Timer",
        "Grammarian and Word of the Day",
        "Ah Counter",
        "Ballot Counter",
        'Sergeant at Arms',
        'Thought of the Day',
        'Stand-in'
    ]
}

all_jobs = [
        job
        for category in jobs.values()
    for job in category
]

In [25]:
all_jobs

['prepared1',
 'prepared2',
 'prepared3',
 'impromptu1',
 'impromptu2',
 'impromptu3',
 'evaluator1',
 'evaluator2',
 'evaluator3',
 'President',
 'Toastmaster',
 'Table Topics Master',
 'General Evaluator',
 'Greeter',
 'Joke Master',
 'Timer',
 'Grammarian and Word of the Day',
 'Ah Counter',
 'Ballot Counter',
 'Sergeant at Arms',
 'Thought of the Day',
 'Stand-in']

In [4]:
model = pl.LpProblem("Toastmasters", pl.LpMaximize)

Model 1

- assign a number to each student
- create a N_jobs x M_days schedule. Some days can have one more or less of the main jobs to finish the class
- All numbers in a row (a single day) must be different. (no student has two jobs on a single day)
- All numbers in a column are preferentially different (no student does a job twice)
- Prepared speakers and evaluators must not be the same (an evaluator / speaker pair doesn't compete more than once --- an evaluator / speaker pair doesn't swap roles later, and they don't speak at the same time

Model 2
- assign a binary to each student-job-day combo
- sum student-jobs (sum over a day) is the number of students. Every student does one job each day.
- Nobody should do the same job more than once.
    - sum days for any student-single job is <=1
- Important Speaker roles must be done
    - sum student-impromptu == 1, ditto for other two speaker roles
- be sure students take a leadership role. Some may do roles more times.
    - sum student-leadership-roles-days >=1

I'm digging model 2. Let's see how it goes.

In [32]:
students = students2025

###############################################################
# Variables
###############

#binary variables.
# schedule[(student, day, job)] is 1 if and only if the student takes that job that day.
schedule = {
    (student,day+1, job):pl.LpVariable(
        f"{student}_{day+1}_{job}",
        0,
        1,
        cat = 'Binary'
    )
    for student in students2025
    for day in range(7)
    for job in all_jobs
}

###############################################################
# Constraints
#################

# each student does a given job only once at most
for student in students:

    

SyntaxError: incomplete input (3794674218.py, line 26)